# Leer archivos XML

El formato de archivo .xml tiene una estructura de árbol, mientras que los DataFrame de `pandas` tienen una estructura de tabla 2D. Por lo tanto, no hay una manera automática de conversión entre los dos formatos. Se debe entender bien la estructura del archivo XML y saber como se quiere mapear estos datos en una tabla bidimensional. Por eso cada conversión XML a DataFrame es un problema diferente.

Abajo podemos ver como es la estructura de un archivo XML, en este caso corresponde a un listado de los árboles de Barcelona.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<llistatArbrat> 
    <arbre> 
        <id>24838</id> 
        <posicioX>430639.138486028</posicioX> 
        <posicioY>4587803.20190948</posicioY> 
        <espaiVerd>Valldaura, Pg. (Llucmajor, Pl. - Fabra i Puig, Pl.)</espaiVerd> 
        <adreca>Pg Valldaura, 184</adreca> 
        <alcada>EXEMPLAR</alcada> 
        <nomCientific>Populus nigra 'Italica'</nomCientific> 
        <nomEsp>Chopo lombardo</nomEsp> 
        <nomCat>Pollancre gavatx</nomCat> 
        <nomEspVeg>EXEMPLAR</nomEspVeg> 
        <amplada></amplada> 
        <plantacioDT></plantacioDT> 
        <posicio>VIARI</posicio> 
        <tipAigua></tipAigua> 
        <tipReg>DIFUSIÓ</tipReg> 
        <tipSuperf>GESPA</tipSuperf> 
        <tipSuport>PARTERRE</tipSuport> 
        <cobertaEscocell></cobertaEscocell> 
        <midaEscocell></midaEscocell> 
        <voraEscocell></voraEscocell> 
    </arbre> 
</llistatArbrat>

Podemos apreciar la estructura de árbol donde las categorías contienen subcategorías. Este archivo es bastante sencillo ya que la categoría principal está bajo la etiqueta `llistatArbrat` que contiene la subcategoría `arbre` y esta a su vez contiene todos los datos en campos definidos por etiquetas.

Veamos como leer este archivo:

In [63]:
from xml.etree import ElementTree as ET
import pandas as pd

tree = ET.parse("../BCNOpenData/work/new2.xml")
root = tree.getroot()

col_name = []
for child in root[0]:
    col_name.append(child.tag)
df = pd.DataFrame(columns=col_name)
df

,id,posicioX,posicioY,espaiVerd,adreca,alcada,nomCientific,nomEsp,nomCat,nomEspVeg,amplada,plantacioDT,posicio,tipAigua,tipReg,tipSuperf,tipSuport,cobertaEscocell,midaEscocell,voraEscocell


In [64]:
index = 0
for child in root:
    row=[]
    for tag in child:
        row.append(tag.text)
    row_series = pd.Series(dict(zip(col_name,row)))      
    row_series.name = index
    index += 1
    df = df.append(row_series)
df.head()

,id,posicioX,posicioY,espaiVerd,adreca,alcada,nomCientific,nomEsp,nomCat,nomEspVeg,amplada,plantacioDT,posicio,tipAigua,tipReg,tipSuperf,tipSuport,cobertaEscocell,midaEscocell,voraEscocell
0,24838,430639.138486028,4587803.20190948,"Valldaura, Pg. (Llucmajor, Pl. - Fabra i Puig,...","Pg Valldaura, 184",EXEMPLAR,Populus nigra 'Italica',Chopo lombardo,Pollancre gavatx,EXEMPLAR,None,None,VIARI,None,DIFUSIÓ,GESPA,PARTERRE,None,None,None
1,24844,430616.306474978,4587806.4110877,"Valldaura, Pg. (Llucmajor, Pl. - Fabra i Puig,...","Pg Valldaura, 180",EXEMPLAR,Populus nigra 'Italica',Chopo lombardo,Pollancre gavatx,EXEMPLAR,None,None,VIARI,None,DIFUSIÓ,GESPA,PARTERRE,None,None,None
2,29604,428535.285878997,4581299.73478935,"Llançà (València - Roma, Av.)","C\ Llança, 37",GRAN,Pinus pinea,Pino piñonero,Pi pinyoner; pi pinyer,TERCERA,None,None,VIARI,None,DIFUSIÓ,SAULÓ AMB VEGETACIÓ,PARTERRE,None,None,None
3,39485,428793.407717021,4581225.64271597,"Joan Miró, Parc","Parc Joan Miró, 0",MITJANA,Pinus pinea,Pino piñonero,Pi pinyoner; pi pinyer,SEGONA,None,None,VIARI,None,MÀNEGA,PAVIMENT,ESCOCELL RODÓ,SENSE COBERTURA,ENTRE 60 I 100 cm,VORA METÀL·LICA
4,42691,433516.857016434,4585967.12222804,"Prim, Rbla. (Guipúscoa - Binefar)","Rbla Prim, 167",None,Brachychiton populneus,Árbol botella,Arbre ampolla,SEGONA,None,None,VIARI,None,MÀNEGA,SAULÓ SENSE VEGETACIÓ,ALTRES,None,None,None


In [66]:
df.size

3159040

In [68]:
import pickle
    
pickle.dump( df, open( "arbres.pkl", "wb" ) )